In [1]:
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import popeye.og as og
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus, resample_stimulus
from popeye import css
from scipy.io import loadmat
import time
import nibabel as nib
from nilearn import plotting
import os
import tqdm
import pickle
import multiprocessing as mp
import time
from ipywidgets import interact, widgets

# Load helper functions
from dataloader import set_paths, load_stimuli, copy_files
%load_ext autoreload
%autoreload 2

In [2]:
# Initialize parameters
params = {}
params['subjID'] = 'JC'
# Got these from Zhengang, and he got it from rsvp_params.txt
params['viewingDistance'] = 83.5 # in cm
params['screenWidth'] = 36.2 # in cm
params['scaleFactor'] = 10
# params['resampleFactor'] = 0.5
params['tr_length'] = 1.3 # in seconds (got this from Aditya, need to check)
params['dtype'] = ctypes.c_int16

p = set_paths(params)

In [3]:
bar, stim_params = load_stimuli(p)
bar = bar[:, :, 0:201]

# bar = resample_stimulus(bar, params['resampleFactor'])

copy_files(p, params)

# Extract number of TRs
func_data = nib.load(p['pRF_func'])
params['nTRs'] = func_data.shape[-1]

Subject folder already exists


In [4]:
# create stimulus object from popeye
stimulus = VisualStimulus(bar,
                          params['viewingDistance'],
                          params['screenWidth'],
                          params['scaleFactor'],
                          params['tr_length'],
                          params['dtype'],
                          'blinear')

In [5]:
# model to fit to
method = 'ss5'
scan_data = nib.load(p['pRF_func']).get_fdata()
brainmask_data = nib.load(p['pRF_brainmask']).get_fdata() != 0
# Resample brainmask if first 2 dimensions are twice the third dimension
if brainmask_data.shape[0] == 2*brainmask_data.shape[2]:
    brainmask_data = brainmask_data[::2, ::2, :]

In [6]:
# Testing only on visual ROIs
# Load visual ROIs
lh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V1.nii.gz')).get_fdata()
lh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V2d.nii.gz')).get_fdata()
lh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3d.nii.gz')).get_fdata()
lh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3AB.nii.gz')).get_fdata()
rh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V1.nii.gz')).get_fdata()
rh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V2d.nii.gz')).get_fdata()
rh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3d.nii.gz')).get_fdata()
rh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3AB.nii.gz')).get_fdata()
# Combine all ROIs using boolean OR
visual_rois = lh_v1 + lh_v2d + lh_v3d + lh_v3ab + rh_v1 + rh_v2d + rh_v3d + rh_v3ab
visual_rois = visual_rois > 0
visual_rois = lh_v1 + rh_v1
visual_rois = visual_rois > 0

In [24]:
slice_x = 82
slice_y = 82
slice_z = 82
scan_plot = scan_data.copy()
scan_plot = scan_plot[:, :, :, 0]
# scan_plot[~brainmask_data] = 0
scan_plot[~visual_rois] = 0
def update_plot(slice_x, slice_y, slice_z):
    plt.figure(figsize=(15, 15))
    # Subplot for slice_x
    plt.subplot(131)
    plt.imshow(scan_plot[:, slice_x, :], cmap='gray', origin='lower')
    plt.title(f'Slice X: {slice_x}')
    plt.axis('off')
    # Subplot for slice_y
    plt.subplot(132)
    plt.imshow(scan_plot[slice_y, :, :], cmap='gray', origin='lower')
    plt.title(f'Slice Y: {slice_y}')
    plt.axis('off')
    # Subplot for slice_z
    plt.subplot(133)
    plt.imshow(scan_plot[:, :, slice_z], cmap='gray', origin='lower')
    plt.title(f'Slice Z: {slice_z}')
    plt.axis('off')
    plt.show()
# Create widgets for each slice
slice_x_widget = widgets.IntSlider(min=0, max=scan_plot.shape[0] - 1, value=slice_x, description='Slice X:')
slice_y_widget = widgets.IntSlider(min=0, max=scan_plot.shape[1] - 1, value=slice_y, description='Slice Y:')
slice_z_widget = widgets.IntSlider(min=0, max=scan_plot.shape[2] - 1, value=slice_z, description='Slice Z:')
# Create an interactive plot
interact(update_plot, slice_x=slice_x_widget, slice_y=slice_y_widget, slice_z=slice_z_widget)

interactive(children=(IntSlider(value=82, description='Slice X:', max=127), IntSlider(value=82, description='S…

<function __main__.update_plot(slice_x, slice_y, slice_z)>

In [7]:
css_model = css.CompressiveSpatialSummationModel(stimulus, utils.spm_hrf)
css_model.hrf_delay = 0
css_model.mask_size = 1

In [8]:
# Create scan data just for visual ROIs
scan_data_visual = scan_data.copy()
scan_data_visual[~visual_rois] = 0

[xi, yi, zi] = np.nonzero(visual_rois)
indices = [(xi[i], yi[i], zi[i]) for i in range(len(xi))]

# set search grid
x_grid = (-5, 5)
y_grid = (-5, 5)
s_grid = (0.25, 2.0)
n_grid = (0.05, 0.5)
grids = (x_grid, y_grid, s_grid, n_grid,)

# set search bounds
x_bounds = (-10, 10)
y_bounds = (-10, 10)
sigma_bounds = (1/css_model.stimulus.ppd, 5.25)
n_bounds = (1e-8, 1)
beta_bounds = (1e-8, None)
baseline_bounds = (None, None)
bounds = (x_bounds, y_bounds, sigma_bounds, n_bounds, beta_bounds, baseline_bounds)

# fit settings
Ns = 3
auto_fit = 1
verbose = 2

bundle = utils.multiprocess_bundle(css.CompressiveSpatialSummationFit, css_model, scan_data_visual, grids, bounds, indices, Ns, auto_fit, verbose)

In [9]:
bundle

[(popeye.css.CompressiveSpatialSummationFit,
  array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
  
         ...,
  
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
     

In [11]:
# Run the fit
start_time = time.time()
ncpu = mp.cpu_count()-1#int(os.environ.get('SLURM_CPUS_PER_TASK',default=1))
print(f"Popeye will analyze {np.sum(visual_rois)} voxels using {ncpu} CPUs")
pool = mp.Pool(ncpu)
output = pool.map(utils.parallel_fit, bundle)
# with mp.Pool(ncpu) as pool:
#     results = list(tqdm.tqdm(pool.imap(utils.parallel_fit, bundle), total=len(bundle), desc='Fitting pRFs'))
pool.close()
pool.join()
end_time = time.time()

Popeye will analyze 2707 voxels using 7 CPUs


KeyboardInterrupt: 